This program reads a tab separated file containing the author name affiliation, and generate the coordinates corresponding to their institude.

One line of the file looks like this:     
`Adam	Y.	Université de Liège	Liège	Belgium`      
The city column was added to make easier the localisation.

For each year, two output files are created:
1. a file storing only the coordinates, that can be used directly with leaflet (see [leaflet](./leaflet) folder) 
2. a file that is the same as the input file but with 2 extra columns for the coordinates.

In [4]:
import os
import glob
import logging
from collections import Counter
from liegecolloquium import Participant
from importlib import reload

# Configuration

In [5]:
datadir = "../data/"
outputdir = "../data/"
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [8]:
filelist = sorted(glob.glob(os.path.join(datadir, "ParticipantList-1973.tsv")))
logging.info("Working on {0} files".format(len(filelist)))

INFO:root:Working on 1 files


In [9]:
participantlist = []
locationlist = []

for datafiles in filelist:
    logging.info("Working on file {0}".format(datafiles))
    outputbasename = os.path.basename(datafiles).replace(".tsv", ".dat")
    coordbasename = os.path.basename(datafiles).replace(".tsv", ".coord")
    outputfile = os.path.join(outputdir, outputbasename)
    coordfile = os.path.join(outputdir, coordbasename)
    logging.info("Writing info in file {0}".format(outputfile))
    logging.info("Writing coordinates in file {0}".format(outputfile))
    
    with open(coordfile, 'w') as cf:
        cf.write("var coords = [\n")

    with open(datafiles, 'r') as f:
        for line in f:
            l = line.rstrip().split('\t')
            
            participant = Participant(l[0], l[1], l[2], l[3], l[4])
            
            # Modify country name via dictionary
            participant.replace_country()
            
            # Find location ; check if in the list before using geopy
            loc1 = ", ".join((participant.affiliation, participant.city, participant.country))
            loc2 = ", ".join((participant.city, participant.country))
            
            if loc1 in locationlist:
                logger.debug("Already in the list")
                ind = locationlist.index(loc1)
                logger.debug(ind)
                participant.location = participantlist[ind].location
            elif loc2 in locationlist:
                logger.debug("Already in the list")
                ind = locationlist.index(loc2)
                logger.debug(ind)
                participant.location = participantlist[ind].location
            else:
                logger.debug("Not in the list, so need to use Nomatim search engine")
                locationstring = participant.get_location()
                locationlist.append(locationstring)
                
            # Write info to a text file
            participant.write_to(outputfile)
            
            # Write coordinates (only) into another file
            participant.write_coords_to(coordfile)
            
            # Append to list
            participantlist.append(participant)
            
    with open(coordfile, 'a') as cf:
        cf.write("]")

INFO:root:Working on file ../data/ParticipantList-1973.tsv
INFO:root:Writing info in file ../data/ParticipantList-1973.dat
INFO:root:Writing coordinates in file ../data/ParticipantList-1973.dat
DEBUG:root:Not in the list, so need to use Nomatim search engine
DEBUG:root:Already in the list
DEBUG:root:0
DEBUG:root:Not in the list, so need to use Nomatim search engine
DEBUG:root:Not in the list, so need to use Nomatim search engine
DEBUG:root:Not in the list, so need to use Nomatim search engine
DEBUG:root:Not in the list, so need to use Nomatim search engine
DEBUG:root:Not in the list, so need to use Nomatim search engine
DEBUG:root:Not in the list, so need to use Nomatim search engine
DEBUG:root:Already in the list
DEBUG:root:0
DEBUG:root:Already in the list
DEBUG:root:0
DEBUG:root:Not in the list, so need to use Nomatim search engine
DEBUG:root:Not in the list, so need to use Nomatim search engine
DEBUG:root:Already in the list
DEBUG:root:0
DEBUG:root:Not in the list, so need to use No